In [1]:
! ls ../../data/

CrowdFlowerAnnotations.txt  Flickr8k.token.txt	       machine_translation
ExpertAnnotations.txt	    Flickr_8k.devImages.txt    readme.txt
Flicker8k_smaller	    Flickr_8k.testImages.txt
Flickr8k.lemma.token.txt    Flickr_8k.trainImages.txt


In [2]:
! pip3 install jieba

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 2.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for jieba: filename=jieba-0.42.1-py3-none-any.whl size=19314477 sha256=ed27d19a84f32bfb6048db6e791987820e2c52697ae582f47ab1202f2dc61d03
  Stored in directory: /root/.cache/pip/wheels/24/aa/17/5bc7c72e9a37990a9620cc3aad0acad1564dcff6dbc2359de3
Successfully built jieba

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Layer
import numpy as np
import random
import jieba

batch_size = 16  
epochs = 35  
latent_dim = 128  # Latent dimensionality of the encoding space.
num_samples = 2000  # Number of samples to train on.
data_path = '../../data/machine_translation/cmn.txt'

input_texts = []
target_texts = []
input_tokens = set()
target_tokens = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

# for line in lines[: min(num_samples, len(lines) - 1)]:
for line in random.sample(lines, num_samples):
    input_text, target_text = line.split('\t')
    
    # We use "\t" as the "start sequence" and "\n" as "end sequence"
    target_text = '\t' + target_text + '\n'
    
    tmp = []
    for token in input_text.split(" "):
        token = token.replace(",", "").replace(".", "").replace("?", "").lower()
        tmp.append(token)
        if token not in input_tokens:
            input_tokens.add(token)
    input_texts.append(tmp)
    
    tmp = []
    for token in jieba.cut(target_text, cut_all=False):
        tmp.append(token)
        if token not in target_tokens:
            target_tokens.add(token)
    target_texts.append(tmp)

input_tokens = list(input_tokens)
target_tokens = list(target_tokens)
num_encoder_tokens = len(input_tokens)
num_decoder_tokens = len(target_tokens)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

print("\ninput data set:")
print(input_texts[:10])
print("\ntarget data set:")
print(target_texts[:10])

2025-06-12 11:21:40.983870: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-06-12 11:21:40.984088: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 3.269 seconds.
Prefix dict has been built successfully.


Number of samples: 2000
Number of unique input tokens: 2162
Number of unique output tokens: 3189
Max sequence length for inputs: 19
Max sequence length for outputs: 24

input data set:
[['a', 'string', 'of', 'disasters', 'struck', 'the', 'region'], ['people', 'are', 'counting', 'on', 'us'], ['a', 'person', 'views', 'things', 'differently', 'according', 'to', 'whether', 'they', 'are', 'rich', 'or', 'poor'], ['i', 'had', 'no', 'choice', 'but', 'to', 'stay', 'in', 'bed', 'all', 'day'], ['thank', 'you', 'all', 'the', 'same'], ['i', "don't", 'remember', 'mailing', 'the', 'letter'], ['he', 'is', 'to', 'come', 'to', 'my', 'house', 'tonight'], ["i'm", 'sorry', 'that', 'you', 'are', 'leaving', 'here'], ['he', 'is', 'no', 'fool'], ['i', 'envied', 'his', 'new', 'house']]

target data set:
[['\t', '一連串', '的', '災難', '襲擊', '這個', '地區', '。', '\n'], ['\t', '人们', '正在', '依赖', '着', '我们', '。', '\n'], ['\t', '每一個', '人', '對', '事情', '的', '看法', '不同', '是', '依據', '他們', '是', '富有', '還是', '貧窮', '。', '\n'], ['\t', '

In [2]:
input_token_index = dict(
    [(token, i) for i, token in enumerate(input_tokens)])
target_token_index = dict(
    [(token, i) for i, token in enumerate(target_tokens)])

print("\ninput index:")
print(input_token_index)

print("\ntarget index:")
print(target_token_index)


input index:
{'worst': 0, "they're": 1, 'sure': 2, 'bought': 3, 'begins': 4, 'tv': 5, 'butterfly': 6, 'fell': 7, 'capitals': 8, 'workers': 9, 'sleepy': 10, 'capture': 11, 'period': 12, 'both': 13, 'lock': 14, 'captain': 15, 'learned': 16, 'your': 17, 'version': 18, 'saxophone': 19, 'engineer': 20, 'id': 21, 'between': 22, 'hate': 23, "today's": 24, 'table"': 25, 'fool': 26, 'asked': 27, 'voice': 28, 'property': 29, 'brave': 30, 'looks': 31, 'major': 32, 'postponed': 33, 'according': 34, 'dirty': 35, 'our': 36, 'fluent': 37, 'yen': 38, 'singing': 39, 'stolen': 40, 'name': 41, 'smokes': 42, 'smile': 43, 'mary': 44, 'tokyo': 45, 'drive': 46, 'nonsense': 47, 'made': 48, 'hating': 49, 'facebook': 50, 'wash': 51, 'moment': 52, "i'd": 53, 'sheet': 54, 'commanded': 55, 'knees': 56, 'ladder': 57, 'business': 58, 'basket': 59, 'start': 60, 'demanded': 61, 'welcome': 62, 'trying': 63, 'glad': 64, 'climbed': 65, 'appetite': 66, 'hill': 67, 'cellular': 68, 'girl': 69, 'fox': 70, 'here': 71, 'uncer

In [3]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, token in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[token]] = 1.
    for t, token in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[token]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[token]] = 1.

print("\nencoder shape:")
print(encoder_input_data.shape)
print("\ndecoder input shape:")
print(decoder_input_data.shape)
print("\ndecoder target shape:")
print(decoder_target_data.shape)
            
print("\nencoder data:")
print(encoder_input_data[-2:])
print("\ndecoder input data:")
print(decoder_input_data[-2:])
print("\ndecoder target data:")
print(decoder_target_data[-2:])


encoder shape:
(2000, 19, 2162)

decoder input shape:
(2000, 24, 3189)

decoder target shape:
(2000, 24, 3189)

encoder data:
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]

decoder input data:
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]

decoder target data:
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0

In [4]:
class Encoder(Model):
    def __init__(self, vocab_size, latent_dim, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.latent_dim = latent_dim
        self.lstm = LSTM(self.latent_dim,
                        return_sequences=True,
                        return_state=True,
                        recurrent_initializer='glorot_uniform')

    def call(self, x):
        output, state_h, state_c = self.lstm(x)
        return output, state_h, state_c


encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = Encoder(num_encoder_tokens, latent_dim, batch_size)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

print(state_h.shape)
print(state_c.shape)
print(encoder_states)
print(encoder_outputs.shape)

2025-06-12 11:22:06.008337: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2025-06-12 11:22:06.033294: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2025-06-12 11:22:06.035364: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2025-06-12 11:22:06.039021: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (eb4c86b6f34a): /proc/driver/nvidia/version does not exist
2025-06-12 11:22:06.064500: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the ap

(None, 128)
(None, 128)
[<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'encoder')>, <KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'encoder')>]
(None, None, 128)


In [12]:
class Decoder(Model):
    def __init__(self, vocab_size, latent_dim, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.latent_dim = latent_dim
        self.lstm = LSTM(latent_dim,
                        return_sequences=True,
                        return_state=True,
                        recurrent_initializer='glorot_uniform')
        self.fc = Dense(vocab_size)

    def call(self, x, hidden):
        # context_vector, attention_weights = self.attention(hidden, enc_output)
        # x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        output, state_h, state_c = self.lstm(x, initial_state=hidden)
        # output = tf.reshape(output, (-1, output.shape[2]))
        y = self.fc(output)

        return y, state_h, state_c 


decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder = Decoder(num_decoder_tokens, latent_dim, batch_size)
decoder_outputs, _, _ = decoder(decoder_inputs, hidden=encoder_states)

print(decoder_outputs)

KerasTensor(type_spec=TensorSpec(shape=(None, None, 3189), dtype=tf.float32, name=None), name='decoder_2/dense_2/BiasAdd:0', description="created by layer 'decoder_2'")


In [13]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 2162)] 0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, None, 3189)] 0                                            
__________________________________________________________________________________________________
encoder (Encoder)               ((None, None, 128),  1172992     input_1[0][0]                    
__________________________________________________________________________________________________
decoder_2 (Decoder)             ((None, None, 3189), 2110197     input_4[0][0]                    
                                                                 encoder[0][1]              

In [14]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
        batch_size=batch_size,
        epochs=15,
        validation_split=0.2)

Epoch 1/15
100/100 [==============================] - 91s 807ms/step - loss: 3.3562 - val_loss: 3.1327
Epoch 2/15
100/100 [==============================] - 106s 1s/step - loss: 3.5979 - val_loss: 3.6821
Epoch 3/15
100/100 [==============================] - 58s 557ms/step - loss: 3.6269 - val_loss: 3.7599
Epoch 4/15
100/100 [==============================] - 49s 495ms/step - loss: 3.5727 - val_loss: 2.9630
Epoch 5/15
100/100 [==============================] - 27s 268ms/step - loss: 3.4254 - val_loss: 3.6227
Epoch 6/15
100/100 [==============================] - 28s 276ms/step - loss: 3.3534 - val_loss: 3.7910
Epoch 7/15
100/100 [==============================] - 26s 263ms/step - loss: 3.3272 - val_loss: 3.2324
Epoch 8/15
100/100 [==============================] - 45s 452ms/step - loss: 3.2567 - val_loss: 3.1741
Epoch 9/15
100/100 [==============================] - 44s 442ms/step - loss: 3.4258 - val_loss: 2.8289
Epoch 10/15
100/100 [==============================] - 55s 543ms/step - los

In [15]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [16]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder(
    decoder_inputs, hidden=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [22]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(10):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: ['a', 'string', 'of', 'disasters', 'struck', 'the', 'region']
Decoded sentence: 了了了了了了了了了了了了了了了了了了了了了了了了了
-
Input sentence: ['people', 'are', 'counting', 'on', 'us']
Decoded sentence: 我。。。。。。。。。。。。。。。。。。。。。。。。
-
Input sentence: ['a', 'person', 'views', 'things', 'differently', 'according', 'to', 'whether', 'they', 'are', 'rich', 'or', 'poor']
Decoded sentence: 了了了了了了了了了了了了了了了了了了了了了了了了了
-
Input sentence: ['i', 'had', 'no', 'choice', 'but', 'to', 'stay', 'in', 'bed', 'all', 'day']
Decoded sentence: 我。。。。。。。。。。。。。。。。。。。。。。。。
-
Input sentence: ['thank', 'you', 'all', 'the', 'same']
Decoded sentence: 他了了了了了了了了了了了了了了了了了了了了了了了了
-
Input sentence: ['i', "don't", 'remember', 'mailing', 'the', 'letter']
Decoded sentence: 我。。。。。。。。。。。。。。。。。。。。。。。。
-
Input sentence: ['he', 'is', 'to', 'come', 'to', 'my', 'house', 'tonight']
Decoded sentence: 了了了了了了了了了了了了了了了了了了了了了了了了了
-
Input sentence: ["i'm", 'sorry', 'that', 'you', 'are', 'leaving', 'here']
Decoded sentence: 我。。。。。。。。。。。。。。。。。。。。